In [ ]:
from transformers import pipeline
from spacy.tokens import Span
import spacy
# Load Hugging Face pipeline
ner_pipeline = pipeline("ner", model="BERT-Style-model/microsoft/deberta-v3-large-4-epoch-8-bs" ,aggregation_strategy = "simple")

text = "user will have username and password"
entities = ner_pipeline(text)
entities

In [ ]:
# Convert Hugging Face output to spaCy format
from spacy import displacy
spacy_ents = {
    "text": text,
    "ents": [
        {"start": ent["start"], "end": ent["end"], "label": ent["entity_group"]}
        for ent in entities
    ],
    "title": "Named Entity Recognition",
}

# Render the visualization
displacy.render(spacy_ents, style="ent", manual=True, jupyter=True)

## Seqeval trial

In [53]:
import pandas as pd
import ast
from transformers import AutoTokenizer, AutoModelForTokenClassification
import torch
from seqeval.metrics import classification_report

# === 1. Load test data ===
df = pd.read_csv("corpus-raymond/test-full-hf-style.csv")  # replace with your path
df["IOB_tag"] = df["IOB_tag"].apply(ast.literal_eval)

# === 2. Load model and tokenizer ===
model_name = "BERT-Style-model/microsoft/deberta-v3-large-4-epoch-8-bs"
tokenizer = AutoTokenizer.from_pretrained(model_name)
model = AutoModelForTokenClassification.from_pretrained(model_name)

# === 3. Define label map ===
# You must match this to your model's label config
id2label = model.config.id2label  # {0: 'O', 1: 'B-ENTITY', ...}
label_map = {v: k for k, v in id2label.items()}  # reverse for later if needed
df["IOB_tag_str"] = df["IOB_tag"].apply(lambda tags: [id2label[tag] for tag in tags])

# === 4. Evaluation ===
y_true = df["IOB_tag_str"]
y_pred = []



In [54]:
for sentence in df["tokens"]:
    encoding = tokenizer(sentence, return_tensors="pt")

    with torch.no_grad():
        logits = model(**encoding).logits

    predictions = torch.argmax(logits, dim=2)  # shape: [seq_len]
    predicted_token_class = [model.config.id2label[t.item()] for t in predictions[0]]
    predicted_token_class = predicted_token_class[1:-1] # remove [pad token]
    y_pred.append(predicted_token_class)

In [55]:
from seqeval.metrics import accuracy_score
from seqeval.metrics import classification_report
from seqeval.metrics import f1_score
f1_score(y_true[0:15], y_pred[0:15])

ValueError: Found input variables with inconsistent numbers of samples:
[18, 28, 17, 23, 22, 27, 31, 28, 15, 27, 21, 6, 19, 12, 17]
[18, 28, 17, 23, 22, 27, 34, 29, 15, 27, 21, 6, 20, 12, 18]

In [56]:
df["tokens"][6]

'for each procedure the patient need to be prepare and supply need to be collect ( e.g. , probe , drill bit , cement , resin , etc . ) .'

In [63]:
len(y_true[6])

31

In [64]:
len(y_pred[6])

34